In [1]:
import pandas as pd
import numpy as np 
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction import text
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, recall_score,confusion_matrix,precision_score,f1_score,classification_report



In [2]:
!pip install imblearn

In [3]:
data = pd.read_csv("labels.csv",index_col = 0)
del data['text_ocr']
data = data.dropna(subset = ['text_corrected'],how='any')
data['text_corrected'] = data['text_corrected'].str.replace('(\W)',' ')  # (\W) where no string present
data['text_corrected']=data['text_corrected'].str.lower()
data['overall_sentiment']=data['overall_sentiment'].str.lower()
#data['overall_sentiment'].where(~(data['overall_sentiment'] == 'very_positive'), other=0, inplace=True)
#data['overall_sentiment'].where(~(data['overall_sentiment'] == 'very_negative'), other=4, inplace=True)
#data['overall_sentiment'].where(~(data['overall_sentiment'] == 'positive'), other=1, inplace=True)
#data['overall_sentiment'].where(~(data['overall_sentiment'] == 'negative'), other=3, inplace=True)
#data['overall_sentiment'].where(~(data['overall_sentiment'] == 'neutral'), other=2, inplace=True)
data['overall_sentiment'] = data['overall_sentiment'].map({'very_positive':1,'positive':2,'negative':3,'very_negative':4,'neutral':0}) 
stop_words = stopwords.words('english')
data['text_corrected'] = data['text_corrected'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
data.tail(10)

C:\Users\Laiba\AppData\Local\Temp\ipykernel_3216\3077926814.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text_corrected'] = data['text_corrected'].str.replace('(\W)',' ')  # (\W) where no string present


,image_name,text_corrected,overall_sentiment
6982,image_6983.jpg,still know nothing 2017,3
6983,image_6984.jpg,create hashtag time explain grab helichrysum h...,2
6984,image_6985.jpg,best around nothing ever gonna keep ralph macchio,1
6985,image_6986.jpg,oils good por cutting poul smell body buribd b...,0
6986,image_6987.jpg,going problems nodding head suggesting oils kn...,2
6987,image_6988.jpg,tuesday mardi gras wednesday valentine friday ...,0
6988,image_6989.jpg,must watch movies 2017 iti chennai memes maana...,0
6989,image_6990.png,less talking planning soda junk food complaini...,2
6990,image_6991.jpg,time fantasy one time unless make time arhtist...,1
6991,image_6992.jpg,starting point every good idea arhtistic licen...,2


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
v=TfidfVectorizer()
x=v.fit_transform(data.text_corrected)
x=x.toarray()
x


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
#BALANCE
from imblearn.over_sampling import RandomOverSampler as ros
x,y=ros().fit_resample(x,data.overall_sentiment)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20)

In [8]:
Model1 = KNeighborsClassifier(n_neighbors=10)
Model1.fit(X_train, y_train)
y_prediction=Model1.predict(X_test)
print ("Accuracy = " , accuracy_score(y_test,y_prediction)*100)  
print("F1 Score = ",f1_score(y_test, y_prediction, average='macro'))
print("Classification Report")
print(classification_report(y_test, y_prediction))
print("confusion_matrix")
print(confusion_matrix(y_prediction,y_test))

Accuracy =  57.28466218379763
F1 Score =  0.4760063819329914
Classification Report
              precision    recall  f1-score   support

           0       0.73      0.05      0.10       623
           1       0.35      0.93      0.51       629
           2       0.00      0.00      0.00       625
           3       0.70      0.88      0.78       634
           4       0.97      1.00      0.99       612

    accuracy                           0.57      3123
   macro avg       0.55      0.57      0.48      3123
weighted avg       0.55      0.57      0.48      3123

confusion_matrix
[[ 33   4   8   0   0]
 [488 584 504  74   0]
 [  0   2   0   0   0]
 [ 94  34 108 560   0]
 [  8   5   5   0 612]]


In [9]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
predicts = nb.predict(X_test)
print(confusion_matrix(y_test,predicts))
print(classification_report(y_test,predicts))
print(accuracy_score(y_test, predicts)) #0.57 0.47

[[290  85 104  81  63]
 [ 45 465  47  43  29]
 [146 149 148 110  72]
 [ 18   8  14 583  11]
 [  0   1   0   2 609]]
              precision    recall  f1-score   support

           0       0.58      0.47      0.52       623
           1       0.66      0.74      0.70       629
           2       0.47      0.24      0.32       625
           3       0.71      0.92      0.80       634
           4       0.78      1.00      0.87       612

    accuracy                           0.67      3123
   macro avg       0.64      0.67      0.64      3123
weighted avg       0.64      0.67      0.64      3123

0.6708293307716939


In [10]:
Text_Model2 = LogisticRegression()
Text_Model2.fit(X_train, y_train)
y_prediction=Text_Model2.predict(X_test)

print ("Accuracy = " , accuracy_score(y_test,y_prediction)*100,"\n")  
print("F1 Score = ",f1_score(y_test, y_prediction, average='macro'),"\n")
print("Classification Report")
print(classification_report(y_test, y_prediction))
print("confusion_matrix ")
print(confusion_matrix(y_prediction,y_test))

Accuracy =  72.55843739993595 

F1 Score =  0.7156852245635292 

Classification Report
              precision    recall  f1-score   support

           0       0.59      0.53      0.56       623
           1       0.71      0.77      0.74       629
           2       0.49      0.39      0.44       625
           3       0.83      0.93      0.87       634
           4       0.94      1.00      0.97       612

    accuracy                           0.73      3123
   macro avg       0.71      0.73      0.72      3123
weighted avg       0.71      0.73      0.72      3123

confusion_matrix 
[[333  49 171  16   0]
 [ 70 487 121  10   0]
 [170  70 246  20   0]
 [ 35  21  68 588   0]
 [ 15   2  19   0 612]]


C:\Users\Laiba\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
#Images
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [12]:
image=[]
path = r"C:\Users\Laiba\Desktop\images"

#slash="\\"

for i in range(data.shape[0]):
    image_path = path+"\\"+data.image_name.iloc[i]
    image_open =Image.open(image_path)
    #imarr=np.array(Image.open(image_path),dtype=np.float)
    newsize = (25, 25)
    image_newsize = image_open.resize(newsize) #resizing an image to the given new size 
    image_array=np.array(image_newsize , dtype=np.float) #creating numpy array of each image provided
    image_array = image_array.flatten() #flatten the image into 1D array
    image_array=np.pad(image_array, len(image_array)%(25*25),"constant") #String padding is useful for output formatting and alignment of strings while printing.
    print(image_array)             
    image.append(image_array)
    if i == 100 :
        break;

C:\Users\Laiba\AppData\Local\Temp\ipykernel_3216\2349510987.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  image_array=np.array(image_newsize , dtype=np.float) #creating numpy array of each image provided


[129.  99.  97. ... 134.  70.  45.]
[255. 255. 255. ... 255. 255. 255.]
[ 26.  40.  72. ... 122. 122. 123.]
[ 13.  10. 188.  13.  24.  23.  23.   3.   3.   3.   1.  50.  35.   9.
  27.   2.  42.  15.   9.  15.   5.  27.  54.   5.  15.  82.  34.  34.
  25.  53. 104.  91. 108.  14. 178.   1.   4.  45.   9.   5.   5.   5.
   5. 120. 122.   9.  15.   5.  15.  15.  34.  25. 221. 225. 179.  29.
  23.  14. 108.   6.  14.  50.  58.   9.   5.  44.  44.  37.  53.  46.
  41.  46.  42.  42.  29.  13. 191.  21. 225. 203.  70.  46.   6. 108.
   6.   1.  28.  45.  65.  65.  36.  20.  96. 129.  52.  52.  41.  20.
  29.  29.  78.  34.  77. 216. 131. 128.  46. 147.  14.  16.  86.  28.
  28.  17.  19.  65.  37.  63.  98.  83. 128. 141. 170. 153. 187. 139.
 139.  33.  53. 144. 151.  42.  94.  11.   6.  57.  28.  28.  56.  17.
  19.  17. 137.  63. 194. 131. 116. 170.  62.  15.  78.  82.  10.  53.
 162. 111. 161.  98.  23.  14.  86.  28.  97.  56.  17.  56.  20.  24.
 192. 200. 194. 132. 129.  80.  93.  18.

In [14]:
# help taken from : https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
digits = datasets.load_digits()

y = data.overall_sentiment[:101]
data1 = image
data1 = pd.Series(data1)

# flatten the images
n_samples = len(digits.images)
data1 = digits.images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data1, digits.target, test_size=0.5, shuffle=False
)

# Learn the digits on the train subset
clf.fit(X_train, y_train)

# Predict the value of the digit on the test subset
predicted = clf.predict(X_test)

In [15]:
Image_Model1 = KNeighborsClassifier(n_neighbors=10)
Image_Model1.fit(X_train, y_train)
y_prediction=Image_Model1.predict(X_test)

print ("Accuracy = " , accuracy_score(y_test,y_prediction)*100)  
print("F1 Score = ",f1_score(y_test, y_prediction, average='macro'))
print("Classification Report = ")
print(classification_report(y_test, y_prediction))
print("confusion_matrix")
print(confusion_matrix(y_prediction,y_test))

Accuracy =  95.43937708565072
F1 Score =  0.954225858939394
Classification Report = 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        88
           1       0.95      0.98      0.96        91
           2       0.98      0.93      0.95        86
           3       0.89      0.91      0.90        91
           4       1.00      0.93      0.97        92
           5       0.95      0.98      0.96        91
           6       0.99      1.00      0.99        91
           7       0.94      1.00      0.97        89
           8       0.96      0.88      0.92        88
           9       0.92      0.93      0.93        92

    accuracy                           0.95       899
   macro avg       0.96      0.95      0.95       899
weighted avg       0.96      0.95      0.95       899

confusion_matrix
[[88  0  1  0  1  0  0  0  0  0]
 [ 0 89  0  0  0  0  0  0  5  0]
 [ 0  0 80  0  0  0  0  0  2  0]
 [ 0  0  5 83  0  0  0  0  2  3]
 [ 0  0

In [16]:
Image_Model_2= LogisticRegression()
Image_Model_2.fit(X_train, y_train)
y_prediction=Image_Model_2.predict(X_test)

print ("Accuracy = " ,accuracy_score(y_test,y_prediction)*100)  
print("F1 Score = ",f1_score(y_test, y_prediction, average='macro'))
print("Classification Report =")
print(classification_report(y_test, y_prediction))
print("confusion_matrix ")
print(confusion_matrix(y_prediction,y_test))

Accuracy =  92.99221357063404
F1 Score =  0.9305027603195832
Classification Report =
              precision    recall  f1-score   support

           0       0.99      0.95      0.97        88
           1       0.94      0.90      0.92        91
           2       0.99      0.98      0.98        86
           3       0.94      0.84      0.88        91
           4       0.98      0.91      0.94        92
           5       0.88      0.95      0.91        91
           6       0.91      0.99      0.95        91
           7       0.98      0.96      0.97        89
           8       0.89      0.90      0.89        88
           9       0.84      0.93      0.89        92

    accuracy                           0.93       899
   macro avg       0.93      0.93      0.93       899
weighted avg       0.93      0.93      0.93       899

confusion_matrix 
[[84  0  0  0  0  0  0  0  0  1]
 [ 0 82  0  0  0  1  1  0  3  0]
 [ 0  0 84  0  0  0  0  0  1  0]
 [ 0  1  2 76  0  0  0  0  0  2]
 [ 1  

C:\Users\Laiba\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
